In [5]:

import sys
print sys.executable
from jupyter_core.paths import jupyter_data_dir
print(jupyter_data_dir())
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split


data_original = pd.read_csv('../data.csv', sep='|')
legit_binaries = data_original[0:41323].drop(['legitimate'], axis=1)
malicious_binaries = data_original[41323::].drop(['legitimate'], axis=1)
dasmalwerk = pd.read_csv('../dasmalwerk_data.csv', sep='|')
frames = [data_original, dasmalwerk]
result = pd.concat(frames)

/usr/bin/python
/Users/xufuou/Library/Jupyter


ImportError: No module named sklearn.ensemble

SyntaxError: invalid syntax (<ipython-input-2-2c530a9741f4>, line 1)

In [65]:
X = result.drop(['Name', 'md5', 'legitimate'], axis=1).values

y = result['legitimate'].values
y
fsel = ExtraTreesClassifier().fit(X, y)

In [66]:
print fsel

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)


In [67]:
fsel.feature_importances_

array([  8.97176911e-02,   1.73647464e-02,   1.05581618e-01,
         7.27910370e-03,   1.16939598e-02,   1.57410782e-03,
         1.37891478e-03,   3.91378834e-04,   1.74582607e-03,
         7.14084553e-04,   1.53785267e-03,   2.90938784e-02,
         5.58135424e-03,   5.83255916e-03,   3.81670407e-02,
         8.85929058e-03,   4.68591892e-03,   2.82556909e-03,
         5.29517896e-02,   1.16020339e-02,   1.85347276e-03,
         4.40875806e-03,   5.78344353e-03,   7.29194764e-02,
         2.28296134e-01,   6.67124436e-03,   2.74516744e-03,
         3.73065172e-04,   1.01029586e-04,   2.88174659e-05,
         5.31718266e-05,   1.38991742e-02,   1.80318104e-02,
         1.16281852e-02,   3.94308158e-02,   1.15565600e-03,
         1.36961080e-03,   1.12745590e-03,   1.05835516e-03,
         1.53943574e-03,   1.25004192e-03,   3.19967830e-03,
         6.50690555e-03,   2.14437977e-03,   1.05884053e-03,
         2.18696727e-03,   2.47624014e-03,   1.15122383e-02,
         3.22450019e-02,

In [68]:
model = SelectFromModel(fsel, prefit=True)

In [69]:
model

SelectFromModel(estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
        norm_order=1, prefit=True, threshold=None)

In [70]:
X_new = model.transform(X)

In [71]:
X_new.shape

(139135, 10)

In [72]:
X.shape

(139135, 54)

In [73]:
X_new.shape[1]

10

In [74]:
nb_features = X_new.shape[1]

In [75]:
indices = np.argsort(fsel.feature_importances_)[::-1][:nb_features]

In [76]:
indices

array([24, 53,  2,  0, 23, 18, 34, 14, 48, 11])

In [77]:
for f in range(nb_features):
    print("%d. feature %s (%f)" % (f + 1, data.columns[2+indices[f]], fsel.feature_importances_[indices[f]]))

1. feature DllCharacteristics (0.228296)
2. feature VersionInformationSize (0.116353)
3. feature Characteristics (0.105582)
4. feature Machine (0.089718)
5. feature Subsystem (0.072919)
6. feature MajorSubsystemVersion (0.052952)
7. feature SectionsMaxEntropy (0.039431)
8. feature MajorOperatingSystemVersion (0.038167)
9. feature ResourcesMaxEntropy (0.032245)
10. feature ImageBase (0.029094)


In [78]:
algorithms = {
        "DecisionTree": DecisionTreeClassifier(max_depth=10),
        "RandomForest": RandomForestClassifier(n_estimators=50),
        "GradientBoosting": GradientBoostingClassifier(n_estimators=50),
        "AdaBoost": AdaBoostClassifier(n_estimators=100),
        "GNB": GaussianNB()
    }


In [79]:
results = {}
X_train, X_test, y_train, y_test = train_test_split(X_new, y ,test_size=0.25)
print("\nNow testing algorithms")
for algo in algorithms:
    clf = algorithms[algo]
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print("%s : %f %%" % (algo, score*100))
    results[algo] = score

winner = max(results, key=results.get)
print('\nWinner algorithm is %s with a %f %% success' % (winner, results[winner]*100))


Now testing algorithms
GNB : 70.443307 %
DecisionTree : 99.082912 %
RandomForest : 99.390524 %
AdaBoost : 98.418813 %
GradientBoosting : 98.763799 %

Winner algorithm is RandomForest with a 99.390524 % success


In [43]:
print 1000000*0.99458159

994581.59
